In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import sqlalchemy as sa

import database as db
from images import copy_images, get_list_of_images_invalid, save_metadata, select_images
from models import County, DataTrustedIdentifier

In [3]:
engine, session=db.connect()
engine.echo=False

2023-04-03 14:47:06,461 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-04-03 14:47:06,461 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-03 14:47:06,463 INFO sqlalchemy.engine.Engine select current_schema()
2023-04-03 14:47:06,463 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-03 14:47:06,464 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-04-03 14:47:06,464 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
list_minimum_image=[5, 10, 20]
list_color=['rgb', 'grayscale']
list_image_size=['256', '400', '512']
path_out = '/home/xandao/Imagens/br_dataset'
list_images_invalid=get_list_of_images_invalid()
list_images_invalid=list_images_invalid['barcode']

query = session.query(County).distinct().all()
state_unaccented_lower, uf_unaccented_lower, county_unaccented_lower=db.get_state_uf_county(query)

In [5]:
for color in list_color:
    for image_size in list_image_size:
        for minimum_image in list_minimum_image:
            list_level = []
            list_path_images_final = []
            list_seq_final = []
            list_count_samples = []

            level=DataTrustedIdentifier.specific_epithet_trusted
            out=os.path.join(path_out, color.upper(), level.name, image_size, str(minimum_image))

            condition = sa.and_(DataTrustedIdentifier.country_trusted.__eq__('Brasil'),
                                level.is_not(None),
                                sa.or_(uf_unaccented_lower, state_unaccented_lower))

            query = db.get_records_group_by_level(condition, level, minimum_image, session)
            print('color: %s image_size: %s minimum_image: %d' % (color.upper(), image_size, minimum_image))
            select_images(color, image_size, list_count_samples, list_level, list_images_invalid, list_path_images_final, list_seq_final, minimum_image, session, query)

            list_f = copy_images(list_level, list_path_images_final, out)
            save_metadata(color, image_size, level, list_count_samples, list_f, list_level, list_path_images_final, list_seq_final, minimum_image, out, session)

/home/xandao/PycharmProjects/piperaceae-sql/database.py:77: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/20/cprf)
  query = session.query(*columns) \


color: RGB image_size: 256 minimum_image: 5
total of levels: 235 total of images: 10514


,levels,count,dst,paths,seq
0,abutiloides,8,/home/xandao/Imagens/br_dataset/RGB/specific_e...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[9416, 10009, 10651, 12428, 25112, 37910, 3847..."
1,aduncum,377,/home/xandao/Imagens/br_dataset/RGB/specific_e...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[7220, 7221, 7222, 7234, 7742, 7757, 8224, 868..."
2,aequale,30,/home/xandao/Imagens/br_dataset/RGB/specific_e...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[8727, 9316, 10163, 11517, 12577, 20200, 21125..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,country_trusted,country,county,state_province,urls
0,9735,Piper,arboreum,Piper,arboreum,Brasil,Brazil,Antonina,Paraná,[https://storage.googleapis.com/cria-zoomify/h...
1,19770,Ottonia,anisum,Piper,anisum,Brasil,Brasil,Guapimirim,Rio de Janeiro,[https://storage.googleapis.com/cria-zoomify/h...
2,40891,Ottonia,taperana,Piper,anisum,Brasil,Brasil,Palmares,Pernambuco,[https://storage.googleapis.com/cria-zoomify/h...


,0
dst,/home/xandao/Imagens/br_dataset/RGB/specific_e...
color,rgb
image_size,256
minimum_image,5
level_name,specific_epithet_trusted
levels,235
total,10514
average,44.74


...

,0
dst,/home/xandao/Imagens/br_dataset/GRAYSCALE/spec...
color,grayscale
image_size,512
minimum_image,10
level_name,specific_epithet_trusted
levels,160
total,9990
average,62.44


color: GRAYSCALE image_size: 512 minimum_image: 20
total of levels: 106 total of images: 9248


,levels,count,dst,paths,seq
0,aduncum,377,/home/xandao/Imagens/br_dataset/GRAYSCALE/spec...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[7220, 7221, 7222, 7234, 7742, 7757, 8224, 868..."
1,aequale,30,/home/xandao/Imagens/br_dataset/GRAYSCALE/spec...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[8727, 9316, 10163, 11517, 12577, 20200, 21125..."
2,alata,81,/home/xandao/Imagens/br_dataset/GRAYSCALE/spec...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[6857, 6861, 6862, 6863, 6864, 7565, 7859, 986..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,country_trusted,country,county,state_province,urls
0,9735,Piper,arboreum,Piper,arboreum,Brasil,Brazil,Antonina,Paraná,[https://storage.googleapis.com/cria-zoomify/h...
1,19770,Ottonia,anisum,Piper,anisum,Brasil,Brasil,Guapimirim,Rio de Janeiro,[https://storage.googleapis.com/cria-zoomify/h...
2,40891,Ottonia,taperana,Piper,anisum,Brasil,Brasil,Palmares,Pernambuco,[https://storage.googleapis.com/cria-zoomify/h...


,0
dst,/home/xandao/Imagens/br_dataset/GRAYSCALE/spec...
color,grayscale
image_size,512
minimum_image,20
level_name,specific_epithet_trusted
levels,106
total,9248
average,87.25


In [6]:
session.close()
engine.dispose()